In [1]:
import sys
import random
import copy
import json
import numpy as np
import scipy as sp
import pandas as pd
import rebound
import reboundx

# Specify which files to open

In [2]:
migration = "UNMigration" #"NoMigration" "UNMigration"
chemdist = "no_overlap" #"binary" "linear" "no_overlap"
galactictides = "off" #"on" "off"
stellarperturb = "off" #"on" "off"

In [3]:
#List of files to open
filenum = [1]

#Number of comets that are simulated
n_comets = 10

#Number of planets that are simulated
n_planets = 4

#Interval at which rebound simulation snapshot is taken
sim_archive_interval = 5e4 #5e5

#Integrator used by simulation
sim_integrator = "mercurius" #"ias15" "whfast "mercurius"

#Distance (in terms of Hill Radii) at which integrator switches to IAS15 to handle close encounters (for Mercurius)
RcriticalMerc = 3.

#Set distance (in AU) at which objects are removed from the sim
object_exit_distance = 3000 #3000 for SD #206266 is 1 pc #1e99 for no removals

sim_name = "5-40 AU Initial Distribution"

#tmax is the total simulation time
tmax = 4.5e9 #1.2e6 #4.5e9

In [4]:
if sim_integrator == "ias15":
    integrator_filelabel = "_IAS15_"
elif sim_integrator == "mercurius":
    integrator_filelabel = "_Rcrit"+str(RcriticalMerc)+"_"
    
if galactictides == "on":
    tides_filelabel = "_Tides"
else:
    tides_filelabel = ""
    
if stellarperturb == "on":
    perturb_filelabel = "_Perturb"
else:
    perturb_filelabel = ""

## Loading simulation and give obtained results

In [9]:
def simulation_load(filenum):
    t_list = []
    #load Saved Simulation archive
    sa = rebound.SimulationArchive("sim_save_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+
                               tides_filelabel+perturb_filelabel+".bin")
    print("sim_save_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+
                               tides_filelabel+perturb_filelabel+".bin")
    print("Number of snapshots: %d" % len(sa))
    print("Time of first and last snapshot: %.1f, %.1f" % (sa.tmin, sa.tmax))
   
    #Read-in chem ratios and hashes
    with open("ChemRatio_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem"+tides_filelabel+
                    perturb_filelabel+".json","r") as chem_file:
        chem_dict = json.load(chem_file)

    for sim in sa:
        t_list.append(sim.t)
    
    print("")
    print(t_list)
    print("")
    if chemdist == "binary" or chemdist == "no_overlap":                
        for sim_saved in sa:
            for non_stellar_object,chem_ratio in chem_dict.items():
                try:
                    print(non_stellar_object+", "+str(sim_saved.particles[non_stellar_object].a)+", "+
                        str(sim_saved.particles[non_stellar_object].e)+", "+str(chem_ratio[0])+", "+str(chem_ratio[1]))
                except:
                    pass
            print("")

    elif chemdist == "linear":        
        for sim_saved in sa:
            for non_stellar_object,chem_ratio in chem_dict.items():
                try:
                    print(non_stellar_object+", "+str(sim_saved.particles[non_stellar_object].a)+", "+
                        str(sim_saved.particles[non_stellar_object].e)+", "+str(chem_ratio))
                except:
                    pass
            print("")

    return sim_name, sa, t_list, chem_dict

# Call Simulations

In [12]:
for file in filenum:
    sim_name, sa, t_list, chem_dict = simulation_load(file)    

sim_save_UNMigration_1_Rcrit3.0_no_overlapchem.bin
Number of snapshots: 241
Time of first and last snapshot: 0.0, 1200000.0

[0.0, 5000.148007962041, 10000.0, 15000.148007951491, 20000.0, 25000.148007982163, 30000.0, 35000.14800798216, 40000.0, 45000.14800798216, 50000.0, 55000.14800798216, 60000.0, 65000.14800798216, 70000.0, 75000.14800785948, 80000.0, 85000.14800785948, 90000.0, 95000.14800785948, 100000.0, 105000.14800785948, 110000.0, 115000.14800785948, 120000.0, 125000.14800785948, 130000.0, 135000.14800785948, 140000.0, 145000.14800785948, 150000.0, 155000.14800785948, 160000.0, 165000.14800785948, 170000.0, 175000.14800785948, 180000.0, 185000.14800785948, 190000.0, 195000.14800785948, 200000.0, 205000.14800785948, 210000.0, 215000.14800785948, 220000.0, 225000.14800785948, 230000.0, 235000.14800785948, 240000.0, 245000.14800785948, 250000.0, 255000.14800785948, 260000.0, 265000.1480078595, 270000.0, 275000.1480078595, 280000.0, 285000.1480078595, 290000.0, 295000.1480078595, 

/home/Michael/anaconda3/lib/python3.6/site-packages/rebound/simulationarchive.py:127: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary file.
  warnings.warn(message, RuntimeWarning)


Comet_8, 38.850490488970735, 0.37764615715591704, 0, 1.0
Comet_9, 30.01668835343489, 0.10058065977206015, 1.0, 0
Comet_10, 33.09374432626181, 0.5227491943792613, 0, 1.0

uranus, 18.238268143579745, 0.14659245700900392, nan, nan
neptune, 28.9994292723807, 0.039808319405272956, nan, nan
Comet_0, 22.448154448903622, 0.36971328675414145, 0, 1.0
Comet_2, 30.890807184000924, 0.055355040540484984, 0, 1.0
Comet_3, 33.99823236743133, 0.3401562429324502, 0, 1.0
Comet_4, 25.87956714749901, 0.2137219185315958, 0, 1.0
Comet_6, 35.402184281914415, 0.27093603851566245, 0, 1.0
Comet_7, 38.3884864624276, 0.10660010367754295, 0, 1.0
Comet_8, 38.48367132014869, 0.3749597528999298, 0, 1.0
Comet_9, 30.00835955331259, 0.07616836775192345, 1.0, 0
Comet_10, 33.01458400716982, 0.5209128500064267, 0, 1.0

uranus, 18.23648691480817, 0.15116946889502572, nan, nan
neptune, 29.069447036276397, 0.03804648060340543, nan, nan
Comet_0, 21.972040813668226, 0.3774567147155282, 0, 1.0
Comet_2, 27.945440231637274, 0.036472

Comet_10, 30.722305281015192, 0.5296089116561031, 0, 1.0
Comet_11, 33.1127137543484, 0.34475003635066903, 0, 1.0
Comet_12, 37.868990607405976, 0.06638472384240618, 0, 1.0

uranus, 18.873379527263417, 0.06341956538851332, nan, nan
neptune, 30.132732902316544, 0.01660156395480313, nan, nan
Comet_0, 21.94462166591031, 0.46559278290743583, 0, 1.0
Comet_2, 23.86630735667926, 0.2725032786517624, 0, 1.0
Comet_3, 41.334686957833135, 0.3966552478634319, 0, 1.0
Comet_6, 27.606955585571143, 0.23675794002025327, 0, 1.0
Comet_7, 37.87710891114347, 0.06672930403974361, 0, 1.0
Comet_10, 30.636349312300474, 0.5274873928180226, 0, 1.0
Comet_11, 33.00856402730561, 0.3431893500574322, 0, 1.0
Comet_12, 37.876962543719316, 0.06671894689004618, 0, 1.0

uranus, 18.891017306424526, 0.059167745953889586, nan, nan
neptune, 30.12118266291473, 0.013429567165916844, nan, nan
Comet_0, 21.947734132753215, 0.46992632641637355, 0, 1.0
Comet_2, 24.27333866745258, 0.2746270456512115, 0, 1.0
Comet_3, 41.32983739785851, 0

uranus, 19.175632761605865, 0.08595157330238686, nan, nan
neptune, 30.483140262531098, 0.025238253844145762, nan, nan
Comet_0, 26.16955170537134, 0.4150508718694219, 0, 1.0
Comet_2, 24.01700183614522, 0.37990492664249, 0, 1.0
Comet_6, 30.765127557229967, 0.10235038416199611, 0, 1.0
Comet_7, 37.8515622161716, 0.055057189817363315, 0, 1.0
Comet_10, 33.58358307799755, 0.541546442997054, 0, 1.0
Comet_12, 37.62838047427337, 0.03657986480788417, 0, 1.0
Comet_13, 37.190130530163145, 0.4321858457796258, 0, 1.0
Comet_14, 29.46888178468025, 0.3405027329441072, 0, 1.0
Comet_15, 25.598805598417194, 0.3822690880933892, 0, 1.0

uranus, 19.133007265774424, 0.09786264422363321, nan, nan
neptune, 30.57457845003971, 0.028828516491855295, nan, nan
Comet_0, 26.26415494110521, 0.4159336682979851, 0, 1.0
Comet_2, 23.761889332916557, 0.3620896544439459, 0, 1.0
Comet_6, 30.306468433053762, 0.10107983675245279, 0, 1.0
Comet_7, 37.8076068261605, 0.05344760043740115, 0, 1.0
Comet_10, 33.50581439676441, 0.5416913